In [0]:
your_storage_account_access_key ="your_key"
spark.conf.set("fs.azure.account.key.kaninipro.dfs.core.windows.net",your_storage_account_access_key)

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.databricks.optimizer.adaptive.enabled", "false")

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
# Define schema
employee_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("joining_year", IntegerType(), False)
])

# Create data (20 records)
employee_data = [
    (1, "Arun", "HR", 2005),
    (2, "Meena", "Finance", 2006),
    (3, "Raj", "IT", 2007),
    (4, "Kavya", "Marketing", 2008),
    (5, "Vikram", "Operations", 2009),
    (6, "Sanjay", "IT", 2010),
    (7, "Divya", "Finance", 2011),
    (8, "Karthik", "HR", 2012),
    (9, "Anu", "Sales", 2013),
    (10, "Prakash", "IT", 2014),
    (11, "Ravi", "Operations", 2015),
    (12, "Priya", "Finance", 2016),
    (13, "Deepak", "IT", 2017),
    (14, "Sneha", "Marketing", 2018),
    (15, "Gopi", "HR", 2019),
    (16, "Shreya", "Finance", 2020),
    (17, "Naveen", "IT", 2021),
    (18, "Latha", "Operations", 2022),
    (19, "Ajay", "Sales",2023),
    (20, "Harini", "Marketing", 2024)
]

# Create data (20 records)
employee_sal_data = [
    (1, 52000.0, 2005),
    (2, 61000.0, 2006),
    (3, 72000.0, 2007),
    (4, 55000.0, 2008),
    (5, 64000.0, 2009),
    (6, 69000.0, 2010),
    (7, 67000.0, 2011),
    (8, 53000.0, 2012),
    (9, 58000.0, 2013),
    (10,74000.0, 2014),
    (11, 65000.0, 2015),
    (12, 71000.0, 2016),
    (13, 76000.0, 2017),
    (14,56000.0, 2018),
    (15, 54000.0, 2019),
    (16, 70000.0, 2020),
    (17, 75000.0, 2021),
    (18, 66000.0, 2022),
    (19, 60000.0, 2023),
    (20, 58000.0, 2024)
]

# Define schema
employee_sal_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("salary", DoubleType(), True),
    StructField("joining_year", IntegerType(), False)
])

# Create DataFrame
employee_data_df = spark.createDataFrame(employee_data, schema=employee_schema).coalesce(1)
employee_sal_data_df = spark.createDataFrame(employee_sal_data, schema=employee_sal_schema).coalesce(1)

spark.sql("drop table if exists tbl_employee_data")

employee_data_df.write.mode("overwrite").format("parquet")\
                                .bucketBy(4, "id","joining_year")\
                                .option("path","abfss://data@kaninipro.dfs.core.windows.net/tbl_employee_data")\
                                .saveAsTable("tbl_employee_data")

spark.sql("drop table if exists tbl_employee_sal_data")
employee_sal_data_df.write.mode("overwrite").format("parquet")\
                                .bucketBy(4, "id","joining_year")\
                                .option("path","abfss://data@kaninipro.dfs.core.windows.net/tbl_employee_sal_data")\
                                .saveAsTable("tbl_employee_sal_data")


In [0]:
emp_df = spark.table("tbl_employee_data")
emp_sal_df = spark.table("tbl_employee_sal_data")

joined_df = emp_df.join(emp_sal_df,["id","joining_year"])

display(joined_df)

#checking parquet file

In [0]:
df = (
    spark.read.format("parquet")
    .load("abfss://data@kaninipro.dfs.core.windows.net/tbl_employee_sal_data")
    .select(
        "*",
        col("_metadata.file_path").alias("file_path"),
        col("_metadata.file_name").alias("file_name"),
        col("_metadata.file_size").alias("file_size_bytes"),
        col("_metadata.file_modification_time").alias("file_modified_time")
    )
)

df.select("file_name").distinct().display()